In [1]:
# when not using colab
import sys
sys.path.append('../src/QA')
import reload_recursive
%reload word_embed
from word_embed import WordEmbed
%reload embed_doc
from embed_doc import EmbedAndConcat

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/max/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
%reload data_prep
from data_prep import DataPrep

In [75]:
%reload model.QA_RNN
from model.QA_RNN import DocumentModel
from model.QA_RNN import QuestionModel

All these functions should be moved to their own module, im just doing it like this as a rough draft.

In [4]:
import torch
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [5]:
df = DataPrep.parse_tsv('../WikiQA-train.tsv')
question_doc_raw_train = DataPrep.convert_pd_to_json(df)

In [6]:
# TODO - add to package somwhere
def get_unrolled_embeddings(df):
    q_inputs = []
    doc_inputs = []
    doc_targets = []
    for key in df.keys():
        toks = DataPrep.tokenize_question_and_doc(df[key])
        q_embeds = EmbedAndConcat.q_concat(toks[0])
        doc_embeds = EmbedAndConcat.doc_concat(toks[1])
        # toks[2] are the targets
        # unroll the document and tokens.
        # that is, create from [[sent1_word1_embed, sent1_word2_embed,...], [sent2_word1_embed...]]
        # simply [word1_embed, word2_embed...]
        unrolled_doc_embeds = [word_embed for sentence in doc_embeds for word_embed in sentence]
        unrolled_doc_targets = [target for sentence in toks[2] for target in sentence]
        q_inputs.append(q_embeds)
        doc_inputs.append(unrolled_doc_embeds)
        doc_targets.append(unrolled_doc_targets)
    return q_inputs, doc_inputs, doc_targets

train_q_inputs, train_doc_inputs, train_doc_targets = get_unrolled_embeddings(question_doc_raw_train)

In [7]:
def get_max(l):
    return max(len(item) for item in l)
def get_min(l):
    return min(len(item) for item in l)

In [8]:
print(get_max(train_q_inputs))
print(get_max(train_doc_inputs))
print(get_max(train_doc_targets))

27
1064
1064


Now do the same for the test set

In [9]:
df_test = DataPrep.parse_tsv('../WikiQA-test.tsv')
question_doc_raw_test = DataPrep.convert_pd_to_json(df_test)
test_q_inputs, test_doc_inputs, test_doc_targets = get_unrolled_embeddings(question_doc_raw_test)

In [10]:
print(get_max(test_q_inputs))
print(get_max(test_doc_inputs))
print(get_max(test_doc_targets))

21
1889
1889


apply padding

In [11]:
import numpy as np
q_max_len = max(get_max(train_q_inputs), get_max(test_q_inputs))
doc_max_len = max(get_max(train_doc_inputs), get_max(test_doc_inputs))

print(f"question max len: {q_max_len}")
print(f"Document max len: {doc_max_len}")
def cut(desired_size, lizt):
    for elt in lizt:
        del elt[desired_size:]

def pad(desired_size, lizt, target=False):
    # embedding size for all should be the same.
    if not target:
        emb_size = np.shape(lizt[0][0])[0]
    for i,_ in enumerate(lizt):
        elt_len = len(lizt[i])
        if elt_len < desired_size:
            if target: # append empty answer to target
                lizt[i].extend(['OOA'] * (desired_size - elt_len))
            else: # pad empty arrays
                lizt[i].extend([np.array([0] * emb_size) for x in range(elt_len, desired_size)])

import copy
train_q_cop = copy.deepcopy(train_q_inputs)
test_q_cop = copy.deepcopy(test_q_inputs)
train_doc_cop = copy.deepcopy(train_doc_inputs)
test_doc_cop = copy.deepcopy(test_doc_inputs)
train_doc_targets_cop = copy.deepcopy(train_doc_targets)
test_doc_targets_cop = copy.deepcopy(test_doc_targets)
pad(q_max_len, train_q_cop)
pad(q_max_len, test_q_cop)
pad(doc_max_len, train_doc_cop)
pad(doc_max_len, test_doc_cop)
pad(doc_max_len, train_doc_targets_cop, target=True)
pad(doc_max_len, test_doc_targets_cop, target=True)
print(f"After padding train question min len {get_min(train_q_cop)}")
print(f"After padding test doc min len {get_min(test_doc_cop)}")

question max len: 27
Document max len: 1889
After padding train question min len 27
After padding test doc min len 1889


In [12]:
# convert the target tags into integers
target2int = {"OOA": 0, "IOA": 1, "BOA": 2, "EOA": 3}
int2target = {0: "OOA", 1: "IOA", 2: "BOA", 3: "EOA"}
for i, targets in enumerate(train_doc_targets_cop):
    for j, _ in enumerate(targets):
        train_doc_targets_cop[i][j] = target2int[train_doc_targets_cop[i][j]]
    
for i, targets in enumerate(test_doc_targets_cop):
    for j, _ in enumerate(targets):
        test_doc_targets_cop[i][j] = target2int[test_doc_targets_cop[i][j]]

In [13]:
from torch.utils.data import TensorDataset
import torch
train_dataset = TensorDataset(torch.Tensor(train_q_cop), torch.Tensor(train_doc_cop), torch.Tensor(train_doc_targets_cop))

from torch.utils.data import DataLoader
#More detailed info about the dataLoader, https://pytorch.org/docs/1.1.0/_modules/torch/utils/data/dataloader.html
batch_size = 128
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True) 
# shuffle (bool, optional) – set to True to have the data reshuffled at every epoch (default: False).

/tmp/ipykernel_4203/2374604515.py:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1670525493953/work/torch/csrc/utils/tensor_new.cpp:230.)
  train_dataset = TensorDataset(torch.Tensor(train_q_cop), torch.Tensor(train_doc_cop), torch.Tensor(train_doc_targets_cop))


In [14]:
q_embed_size = np.shape(train_q_cop[0][0])[0]
doc_embed_size = np.shape(train_doc_cop[0][0])[0]

In [15]:
del train_q_inputs
del test_q_inputs
del train_doc_inputs
del test_doc_inputs
del train_doc_targets
del test_doc_targets
del train_q_cop
del test_q_cop
del train_doc_cop
del test_doc_cop
del train_doc_targets_cop
del test_doc_targets_cop

In [16]:
import gc
gc.collect()

0

In [76]:
doc_rnn_model = QA_RNN.DocumentModel(doc_embed_size, doc_embed_size, doc_embed_size)
question_rnn_model = QA_RNN.QuestionModel(q_embed_size, q_embed_size, doc_embed_size)

In [77]:
def train(input_question_tensor, input_document_tensor, target_tensor, question_model, document_model, question_optimizer, document_optimizer, criterion):
    loss = 0    
    question_optimizer.zero_grad()
    document_optimizer.zero_grad()

    # get output from question model
    question_output = question_model(input_question_tensor)

    # predictions from the document model with output from question for attention
    document_output = document_model(input_document_tensor, question_output)

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [78]:
import time
import math

# Helper functions for training
def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [79]:
import random
def trainIters(question_model, document_model, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    question_model_optimizer = optim.SGD(question_model.parameters(), lr=learning_rate)
    document_model_optimizer = optim.SGD(document_model.parameters(), lr=learning_rate)
    
    criterion = nn.NLLLoss()

    for question_input,doc_input,targets in train_loader:
        question_input = question_input.to(device)
        doc_input = doc_input.to(device)
        targets = targets.to(device)
        
        print("qweqweq")
        loss = train(question_input, doc_input, targets, question_model, document_model, question_model_optimizer, document_model_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0


In [80]:
trainIters(question_rnn_model, doc_rnn_model, 10)

qweqweq
torch.Size([128, 102])
torch.Size([128, 102])
torch.Size([128, 1, 1])
torch.Size([1, 128, 102])


RuntimeError: shape '[128, 1, 1]' is invalid for input of size 13056